In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score


In [2]:
# Load the dataset
df = pd.read_csv('trainhackathon.csv')
df_test=pd.read_csv('Testhackathon.csv')
df.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,dcf68cc2fb515ccad7d8b9b3bd80ee2a4b270063,SAINT-LOUIS,K > 24 month,17000.0,32.0,18000.0,6000.0,34.0,NaN,97.0,355.0,6.0,NaN,NaN,NO,62,All-net 500F=2000F;5d,35.0,0
1,71c44b5ba328db5c4192a80f7cf8f244d9350ed0,NaN,K > 24 month,4300.0,29.0,4427.0,1476.0,37.0,1764.0,8.0,3.0,0.0,NaN,2.0,NO,40,"Data: 100 F=40MB,24H",22.0,0
2,ce46411b1526c94f20a383b8cb188f8d27f82a0a,TAMBACOUNDA,K > 24 month,1500.0,3.0,1500.0,500.0,3.0,NaN,30.0,30.0,NaN,NaN,NaN,NO,32,All-net 500F=2000F;5d,3.0,0
3,f467cdb6669818373c26c2bad44e01ba66f97d21,FATICK,K > 24 month,1500.0,3.0,2497.0,832.0,4.0,0.0,159.0,45.0,19.0,NaN,NaN,NO,18,On net 200F=Unlimited _call24H,3.0,0
4,ec45e1a1888a32b5dcce0954cfec20c6e037db31,FATICK,K > 24 month,NaN,NaN,498.0,166.0,3.0,1.0,1.0,3.0,NaN,NaN,NaN,NO,50,NaN,NaN,0


In [3]:
 df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         400000 non-null  object 
 1   REGION          242480 non-null  object 
 2   TENURE          400000 non-null  object 
 3   MONTANT         259723 non-null  float64
 4   FREQUENCE_RECH  259723 non-null  float64
 5   REVENUE         265337 non-null  float64
 6   ARPU_SEGMENT    265337 non-null  float64
 7   FREQUENCE       265337 non-null  float64
 8   DATA_VOLUME     203146 non-null  float64
 9   ON_NET          254181 non-null  float64
 10  ORANGE          233683 non-null  float64
 11  TIGO            160614 non-null  float64
 12  ZONE1           31690 non-null   float64
 13  ZONE2           25513 non-null   float64
 14  MRG             400000 non-null  object 
 15  REGULARITY      400000 non-null  int64  
 16  TOP_PACK        232671 non-null  object 
 17  FREQ_TOP_P

In [4]:
df.shape

(400000, 19)

In [5]:
num = ['MONTANT', 'FREQUENCE_RECH', 'REVENUE', 'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'REGULARITY',
                      'ON_NET', 'ORANGE', 'TIGO', 'ZONE1', 'ZONE2', 'FREQ_TOP_PACK']
cat= ['REGION', 'TENURE', 'MRG', 'TOP_PACK']


In [6]:
# Fill missing values for numerical columns with the mean
df[num] = df[num].fillna(df[num].mean())

# Fill missing values for categorical columns with the mode
df[cat] = df[cat].apply(lambda x: x.fillna(x.mode()[0]))
df.isna().sum()

user_id           0
REGION            0
TENURE            0
MONTANT           0
FREQUENCE_RECH    0
REVENUE           0
ARPU_SEGMENT      0
FREQUENCE         0
DATA_VOLUME       0
ON_NET            0
ORANGE            0
TIGO              0
ZONE1             0
ZONE2             0
MRG               0
REGULARITY        0
TOP_PACK          0
FREQ_TOP_PACK     0
CHURN             0
dtype: int64

In [7]:
# Fill missing values for numerical columns with the mean
df_test[num] = df_test[num].fillna(df_test[num].mean())

# Fill missing values for categorical columns with the mode
df_test[cat] = df_test[cat].apply(lambda x: x.fillna(x.mode()[0]))
df_test.isna().sum()

user_id           0
REGION            0
TENURE            0
MONTANT           0
FREQUENCE_RECH    0
REVENUE           0
ARPU_SEGMENT      0
FREQUENCE         0
DATA_VOLUME       0
ON_NET            0
ORANGE            0
TIGO              0
ZONE1             0
ZONE2             0
MRG               0
REGULARITY        0
TOP_PACK          0
FREQ_TOP_PACK     0
dtype: int64

In [8]:
df.describe()

,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK,CHURN
count,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.00000
mean,5522.971346,11.503733,5505.487757,1835.167658,13.951835,3369.763441,275.917586,95.532927,23.134608,7.874282,7.187003,28.046502,9.254209,0.18711
std,5720.859508,10.697351,5844.388871,1948.125688,11.956196,8752.618996,696.386587,156.484600,39.960208,10.032222,6.809694,22.282773,9.385172,0.39000
min,20.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.00000
25%,2000.000000,4.000000,1900.000000,633.000000,6.000000,240.000000,12.000000,20.000000,11.000000,7.874282,7.187003,6.000000,4.000000,0.00000
50%,5522.971346,11.503733,5505.487757,1835.167658,13.951835,3369.763441,209.000000,95.532927,23.134608,7.874282,7.187003,24.000000,9.254209,0.00000
75%,5522.971346,11.503733,5505.487757,1835.167658,13.951835,3369.763441,275.917586,95.532927,23.134608,7.874282,7.187003,51.000000,9.254209,0.00000
max,226550.000000,133.000000,233413.000000,77804.000000,91.000000,934576.000000,45011.000000,6788.000000,2758.000000,1657.000000,1011.000000,62.000000,629.000000,1.00000


In [9]:
X = df.drop(columns=['user_id', 'CHURN'])
y = df['CHURN']
X_test1=df_test.drop(columns=['user_id'])

In [10]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [11]:
preprocessor = ColumnTransformer(
    transformers=[('cat', categorical_transformer, cat)])


In [26]:
# Define a simplified model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=111))
])

In [27]:
# Cross-validation to evaluate the model
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_log_loss')
mean_cv_score = -cv_scores.mean()


In [15]:
model.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['REGION', 'TENURE', 'MRG',
                                                   'TOP_PACK'])])),
                ('classifier',
                 LogisticRegression(max_iter=1000, random_state=1111))])

In [16]:
y_test1 = model.predict_proba(X_test1)[:, 1]
final_sub= pd.DataFrame({'user_id': df_test['user_id'], 'CHURN': y_test1})
final_sub['CHURN']=final_sub['CHURN'].round(2)
final_sub.head(5)


,user_id,CHURN
0,af900d87e73b7ff6509d2203df4704a98aa5f2a6,0.40
1,5335efd940280b82143272275637d1e65d37eadb,0.40
2,a581f4fa08677c26f83f643248c667e241043086,0.07
3,64f67177d0775262b8087a9e2e3b8061b6324ae6,0.07
4,0d6009a4594c4be22449b8d9cc01a0bcea98faea,0.40


In [25]:
import os  
os.makedirs('Downloads/DSN_HACKATHON 5', exist_ok=True)
final_sub.to_csv('Downloads/DSN_HACKATHON 5.csv') 

In [18]:
mean_cv_score 

0.36236121311598224

In [24]:
final=check.drop(columns=['Unnamed: 0'])
final.head()

,user_id,CHURN
0,af900d87e73b7ff6509d2203df4704a98aa5f2a6,0.40
1,5335efd940280b82143272275637d1e65d37eadb,0.40
2,a581f4fa08677c26f83f643248c667e241043086,0.07
3,64f67177d0775262b8087a9e2e3b8061b6324ae6,0.07
4,0d6009a4594c4be22449b8d9cc01a0bcea98faea,0.40
